In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
# detection
!pip install --no-deps ../input/siimwbf/ensemble_boxes-1.0.4-py3-none-any.whl

# mmdet install
!pip install ../input/mmdetectionv2130/src/addict-2.4.0-py3-none-any.whl
!pip install ../input/mmdetectionv2130/src/yapf-0.31.0-py2.py3-none-any.whl
!pip install ../input/mmdetectionv2130/src/mmcv_full-1.3.7-cp37-cp37m-manylinux1_x86_64.whl
!rsync -a ../input/mmdetectionv2130/mmdetection ../
!pip install ../input/mmdetectionv2130/src/mmpycocotools-12.0.3
!pip install ../input/pycocotools202/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl
!cd ../mmdetection && pip install -e .


!mkdir /workspace
!rsync -a ../input/siim-mmdet /workspace
!mv /workspace/siim-mmdet /workspace/customized_mmdetection

In [ ]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
if df.shape[0] == 2477:
    is_public = True
    
    fast_sub = True
    if fast_sub:
        fast_df = pd.DataFrame(([['00086460a852_study', 'negative 1 0 0 1 1'], 
                             ['000c9c05fd14_study', 'negative 1 0 0 1 1'], 
                             ['65761e66de9f_image', 'none 1 0 0 1 1'], 
                             ['51759b5579bc_image', 'none 1 0 0 1 1']]), 
                           columns=['id', 'PredictionString'])
    
else:
    fast_sub = False
    is_public = False
    sub_public = pd.read_csv('/kaggle/input/siim-public/sub145/sub.csv')
    pub_ids = sub_public["id"].values
    
    df = df[~df["id"].isin(pub_ids)].reset_index(drop=True)

# .dcm to .png

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
split = 'test'
save_dir = f'/kaggle/tmp/{split}/'

In [ ]:
image_id = []
dim0 = []
dim1 = []
splits = []

# image to study dict, key image_id: item study_id
dict_image2study = {}

img_size = 640

save_dir = f'/kaggle/tmp/{split}/image/'
os.makedirs(save_dir, exist_ok=True)
if fast_sub:
    xray = read_xray('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=img_size)  
    im.save(os.path.join(save_dir,'65761e66de9f_image.png'))
    image_id.append('65761e66de9f.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
    dict_image2study['65761e66de9f_image'] = '00086460a852' + '_study'
    
    xray = read_xray('../input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=img_size)  
    im.save(os.path.join(save_dir, '51759b5579bc_image.png'))
    image_id.append('51759b5579bc.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
    dict_image2study['51759b5579bc_image'] = '000c9c05fd14' + '_study'
else:
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=img_size)  
            im.save(os.path.join(save_dir, file.replace('.dcm', '_image.png')))
            image_id.append(file.replace('.dcm', ''))
            dim0.append(xray.shape[0])
            dim1.append(xray.shape[1])
            splits.append(split)
            dict_image2study[file.replace('.dcm', '_image')] = dirname.split('/')[-2] + '_study'
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})

# study predict

In [ ]:
import numpy as np 
import pandas as pd
if fast_sub:
    df = fast_df.copy()
else:
    df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    if is_public == False:
        df = df[~df["id"].isin(pub_ids)].reset_index(drop=True)
id_laststr_list  = []
for i in range(df.shape[0]):
    id_laststr_list.append(df.loc[i,'id'][-1])
df['id_last_str'] = id_laststr_list

study_len = df[df['id_last_str'] == 'y'].shape[0]

In [ ]:
!pip install /kaggle/input/siim-rsna-library/timm/timm-0.4.9-py3-none-any.whl -q
import timm

import os

import matplotlib.pyplot as plt
import seaborn as sns
import random
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import cv2

import albumentations
from albumentations.pytorch import ToTensorV2

In [ ]:
BATCH_SIZE = 8
NTHREAD = 2

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
# ==========================================================
# Model
# ==========================================================

class Net414(nn.Module):
    def __init__(self, name="resnest101e"):
        super(Net414, self).__init__()
        enet = timm.create_model(name, False, drop_path_rate=0.2)
        self.b0 = nn.Sequential(
            enet.conv_stem,
            enet.bn1,
            enet.act1,
        )
        self.b1 = enet.blocks[0]
        self.b2 = enet.blocks[1]
        self.b3 = enet.blocks[2]
        self.b4 = enet.blocks[3]
        self.b5 = enet.blocks[4]
        self.b6 = enet.blocks[5]
        self.b7 = enet.blocks[6]

        self.after_blocks = nn.Sequential(
            enet.conv_head,
            enet.bn2,
            enet.act2
        )

        self.global_pool = enet.global_pool
        self.dropout = nn.Dropout(0.5)
        self.myfc = nn.Linear(enet.classifier.in_features, 5)
        # self.enet.classifier = nn.Identity()

        # ===============================
        # Mask netfork after b5
        # ===============================
        self.mask = nn.Sequential(
            # nn.Conv2d(176, 128, kernel_size=3, padding=1),
            nn.Conv2d(304, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    def forward(self, x, is_mask=False):
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        x = self.b6(x)
        # =================================
        # Mask net
        # =================================
        mask = self.mask(x)

        x = self.b7(x)

        x = self.after_blocks(x)
        x = self.global_pool(x)
        h = self.myfc(self.dropout(x))
        if is_mask:
            return h, mask
        else:
            return h


class Net415(nn.Module):
    def __init__(self, name="resnest101e"):
        super(Net415, self).__init__()
        enet = timm.create_model(name, False, drop_path_rate=0.2)
        self.b0 = nn.Sequential(
            enet.conv_stem,
            enet.bn1,
            enet.act1,
        )
        self.b1 = enet.blocks[0]
        self.b2 = enet.blocks[1]
        self.b3 = enet.blocks[2]
        self.b4 = enet.blocks[3]
        self.b5 = enet.blocks[4]
        self.b6 = enet.blocks[5]
        self.b7 = enet.blocks[6]

        self.after_blocks = nn.Sequential(
            enet.conv_head,
            enet.bn2,
            enet.act2
        )

        self.global_pool = enet.global_pool
        self.dropout = nn.Dropout(0.5)
        self.myfc = nn.Linear(enet.classifier.in_features, 5)
        # self.enet.classifier = nn.Identity()

        # ===============================
        # Mask netfork after b5
        # ===============================
        self.mask = nn.Sequential(
            # nn.Conv2d(176, 128, kernel_size=3, padding=1),
            nn.Conv2d(512, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    def forward(self, x, is_mask=False):
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        x = self.b6(x)
        x = self.b7(x)
        
        # =================================
        # Mask net
        # =================================
        mask = self.mask(x)

        x = self.after_blocks(x)
        x = self.global_pool(x)
        h = self.myfc(self.dropout(x))
        if is_mask:
            return h, mask
        else:
            return h



class Net416(nn.Module):
    def __init__(self, name="resnest101e"):
        super(Net416, self).__init__()
        enet = timm.create_model(name, False, drop_path_rate=0.2)
        self.b0 = nn.Sequential(
            enet.conv_stem,
            enet.bn1,
            enet.act1,
        )
        self.b1 = enet.blocks[0]
        self.b2 = enet.blocks[1]
        self.b3 = enet.blocks[2]
        self.b4 = enet.blocks[3]
        self.b5 = enet.blocks[4]
        self.b6 = enet.blocks[5]
        self.b7 = enet.blocks[6]

        self.after_blocks = nn.Sequential(
            enet.conv_head,
            enet.bn2,
            enet.act2
        )

        self.global_pool = enet.global_pool
        self.dropout = nn.Dropout(0.5)
        self.myfc = nn.Linear(enet.classifier.in_features, 5)
        # self.enet.classifier = nn.Identity()

        # ===============================
        # Mask netfork
        # ===============================
        self.mask6 = nn.Sequential(
            # nn.Conv2d(176, 128, kernel_size=3, padding=1),
            nn.Conv2d(304, 128, kernel_size=3, padding=1),
            # nn.Conv2d(512, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )

        self.mask7 = nn.Sequential(
            # nn.Conv2d(176, 128, kernel_size=3, padding=1),
            # nn.Conv2d(304, 128, kernel_size=3, padding=1),
            nn.Conv2d(512, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    def forward(self, x, is_mask=False):
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        x = self.b6(x)
        # =================================
        # Mask net
        # =================================
        mask6 = self.mask6(x)

        x = self.b7(x)
        # =================================
        # Mask net
        # =================================
        mask7 = self.mask7(x)

        x = self.after_blocks(x)
        x = self.global_pool(x)
        h = self.myfc(self.dropout(x))
        if is_mask:
            return h, mask6, mask7
        else:
            return h



class Net417(nn.Module):
    def __init__(self, name="resnest101e"):
        super(Net417, self).__init__()
        enet = timm.create_model(name, False, drop_path_rate=0.2)
        self.b0 = nn.Sequential(
            enet.conv_stem,
            enet.bn1,
            enet.act1,
        )
        self.b1 = enet.blocks[0]
        self.b2 = enet.blocks[1]
        self.b3 = enet.blocks[2]
        self.b4 = enet.blocks[3]
        self.b5 = enet.blocks[4]
        self.b6 = enet.blocks[5]
        self.b7 = enet.blocks[6]

        self.after_blocks = nn.Sequential(
            enet.conv_head,
            enet.bn2,
            enet.act2
        )

        self.global_pool = enet.global_pool
        self.dropout = nn.Dropout(0.5)
        self.myfc = nn.Linear(enet.classifier.in_features, 5)
        # self.enet.classifier = nn.Identity()

        # ===============================
        # Mask netfork
        # ===============================
        self.mask5 = nn.Sequential(
            nn.Conv2d(176, 128, kernel_size=3, padding=1),
            # nn.Conv2d(304, 128, kernel_size=3, padding=1),
            # nn.Conv2d(512, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )

        self.mask6 = nn.Sequential(
            # nn.Conv2d(176, 128, kernel_size=3, padding=1),
            nn.Conv2d(304, 128, kernel_size=3, padding=1),
            # nn.Conv2d(512, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )

        self.mask7 = nn.Sequential(
            # nn.Conv2d(176, 128, kernel_size=3, padding=1),
            # nn.Conv2d(304, 128, kernel_size=3, padding=1),
            nn.Conv2d(512, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    def forward(self, x, is_mask=False):
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        # =================================
        # Mask net
        # =================================
        mask5 = self.mask5(x)

        x = self.b6(x)
        # =================================
        # Mask net
        # =================================
        mask6 = self.mask6(x)

        x = self.b7(x)
        # =================================
        # Mask net
        # =================================
        mask7 = self.mask7(x)

        x = self.after_blocks(x)
        x = self.global_pool(x)
        h = self.myfc(self.dropout(x))
        if is_mask:
            return h, mask5, mask6, mask7
        else:
            return h








        
class Net418(nn.Module):
    def __init__(self, name="resnest101e"):
        super(Net418, self).__init__()
        enet = timm.create_model(name, False, drop_path_rate=0.2)
        self.b0 = nn.Sequential(
            enet.conv_stem,
            enet.bn1,
            enet.act1,
        )
        self.b1 = enet.blocks[0]
        self.b2 = enet.blocks[1]
        self.b3 = enet.blocks[2]
        self.b4 = enet.blocks[3]
        self.b5 = enet.blocks[4]
        self.b6 = enet.blocks[5]
        self.b7 = enet.blocks[6]

        self.after_blocks = nn.Sequential(
            enet.conv_head,
            enet.bn2,
            enet.act2
        )

        self.global_pool = enet.global_pool
        self.dropout = nn.Dropout(0.5)
        self.myfc = nn.Linear(enet.classifier.in_features, 5)
        # self.enet.classifier = nn.Identity()

        # ===============================
        # Mask netfork
        # ===============================
        self.mask = nn.Sequential(
            # nn.Conv2d(176, 128, kernel_size=3, padding=1),
            # nn.Conv2d(304, 128, kernel_size=3, padding=1),
            nn.Conv2d(384, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    def forward(self, x, is_mask=False):
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        x = self.b6(x)
        # =================================
        # Mask net
        # =================================
        mask = self.mask(x)
        
        x = self.b7(x)

        x = self.after_blocks(x)
        x = self.global_pool(x)
        h = self.myfc(self.dropout(x))
        if is_mask:
            return h, mask
        else:
            return h


class Net419(nn.Module):
    def __init__(self, name="resnest101e"):
        super(Net419, self).__init__()
        enet = timm.create_model(name, False, drop_path_rate=0.2)
        self.b0 = nn.Sequential(
            enet.conv_stem,
            enet.bn1,
            enet.act1,
        )
        self.b1 = enet.blocks[0]
        self.b2 = enet.blocks[1]
        self.b3 = enet.blocks[2]
        self.b4 = enet.blocks[3]
        self.b5 = enet.blocks[4]
        self.b6 = enet.blocks[5]
        self.b7 = enet.blocks[6]

        self.after_blocks = nn.Sequential(
            enet.conv_head,
            enet.bn2,
            enet.act2
        )

        self.global_pool = enet.global_pool
        self.dropout = nn.Dropout(0.5)
        self.myfc = nn.Linear(enet.classifier.in_features, 5)
        # self.enet.classifier = nn.Identity()

        # ===============================
        # Mask netfork
        # ===============================
        self.mask = nn.Sequential(
            # nn.Conv2d(176, 128, kernel_size=3, padding=1),
            # nn.Conv2d(304, 128, kernel_size=3, padding=1),
            nn.Conv2d(640, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    def forward(self, x, is_mask=False):
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        x = self.b6(x)
        x = self.b7(x)
        # =================================
        # Mask net
        # =================================
        mask = self.mask(x)

        x = self.after_blocks(x)
        x = self.global_pool(x)
        h = self.myfc(self.dropout(x))
        if is_mask:
            return h, mask
        else:
            return h
        


class Net420(nn.Module):
    def __init__(self, name="resnest101e"):
        super(Net420, self).__init__()
        enet = timm.create_model(name, False, drop_path_rate=0.2)
        self.b0 = nn.Sequential(
            enet.conv_stem,
            enet.bn1,
            enet.act1,
        )
        self.b1 = enet.blocks[0]
        self.b2 = enet.blocks[1]
        self.b3 = enet.blocks[2]
        self.b4 = enet.blocks[3]
        self.b5 = enet.blocks[4]
        self.b6 = enet.blocks[5]
        self.b7 = enet.blocks[6]

        self.after_blocks = nn.Sequential(
            enet.conv_head,
            enet.bn2,
            enet.act2
        )

        self.global_pool = enet.global_pool
        self.dropout = nn.Dropout(0.5)
        self.myfc = nn.Linear(enet.classifier.in_features, 5)
        # self.enet.classifier = nn.Identity()

        # ===============================
        # Mask netfork
        # ===============================
        self.mask6 = nn.Sequential(
            # nn.Conv2d(176, 128, kernel_size=3, padding=1),
            nn.Conv2d(384, 128, kernel_size=3, padding=1),
            # nn.Conv2d(512, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )

        self.mask7 = nn.Sequential(
            # nn.Conv2d(176, 128, kernel_size=3, padding=1),
            # nn.Conv2d(304, 128, kernel_size=3, padding=1),
            nn.Conv2d(640, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    def forward(self, x, is_mask=False):
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        x = self.b6(x)
        # =================================
        # Mask net
        # =================================
        mask6 = self.mask6(x)

        x = self.b7(x)
        # =================================
        # Mask net
        # =================================
        mask7 = self.mask7(x)

        x = self.after_blocks(x)
        x = self.global_pool(x)
        h = self.myfc(self.dropout(x))
        if is_mask:
            return h, mask6, mask7
        else:
            return h
        
        

class Net553(nn.Module):
    def __init__(self, name="resnest101e"):
        super(Net553, self).__init__()
        enet = timm.create_model(name, False, drop_path_rate=0.2)
        self.b0 = nn.Sequential(
            enet.conv_stem,
            enet.bn1,
            enet.act1,
        )
        self.b1 = enet.blocks[0]
        self.b2 = enet.blocks[1]
        self.b3 = enet.blocks[2]
        self.b4 = enet.blocks[3]
        self.b5 = enet.blocks[4]
        self.b6 = enet.blocks[5]
        self.b7 = enet.blocks[6]

        self.after_blocks = nn.Sequential(
            enet.conv_head,
            enet.bn2,
            enet.act2
        )

        self.global_pool = enet.global_pool
        self.dropout = nn.Dropout(0.5)
        self.myfc = nn.Linear(enet.classifier.in_features, 5)
        # self.enet.classifier = nn.Identity()

        # ===============================
        # Mask netfork after b5
        # ===============================
        self.mask1 = nn.Sequential(
            # nn.Conv2d(176, 128, kernel_size=3, padding=1),
            # nn.Conv2d(304, 128, kernel_size=3, padding=1),
            nn.Conv2d(224, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )

        self.mask2 = nn.Sequential(
            # nn.Conv2d(176, 128, kernel_size=3, padding=1),
            # nn.Conv2d(304, 128, kernel_size=3, padding=1),
            nn.Conv2d(384, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )

        self.mask3 = nn.Sequential(
            # nn.Conv2d(176, 128, kernel_size=3, padding=1),
            # nn.Conv2d(304, 128, kernel_size=3, padding=1),
            nn.Conv2d(640, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    def forward(self, x, is_mask=False):
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        # =================================
        # Mask net
        # =================================
        mask1 = self.mask1(x)

        x = self.b6(x)
        # =================================
        # Mask net
        # =================================
        mask2 = self.mask2(x)

        x = self.b7(x)
        # =================================
        # Mask net
        # =================================
        mask3 = self.mask3(x)

        x = self.after_blocks(x)
        x = self.global_pool(x)
        h = self.myfc(self.dropout(x))
        if is_mask:
            return h, mask1, mask2, mask3
        else:
            return h
        
        
        
        
        
        
        
        
        
        
        
        


class Net604(nn.Module):
    def __init__(self, name="resnest101e"):
        super(Net604, self).__init__()
        swin = timm.create_model(name, False)

        self.patch_embed = swin.patch_embed
        self.pos_drop = swin.pos_drop
        self.b0 = swin.layers[0]
        self.b1 = swin.layers[1]
        self.b2 = swin.layers[2]
        self.b3 = swin.layers[3]
        self.norm = swin.norm

        self.global_pool = swin.avgpool
        self.dropout = nn.Dropout(0.5)
        self.myfc = nn.Linear(swin.head.in_features, 5)
        # self.enet.classifier = nn.Identity()

        # ===============================
        # Mask netfork after layer16
        # ===============================
        self.mask = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    def forward(self, x, is_mask=False):
        x = self.patch_embed(x)
        x = self.pos_drop(x)
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.norm(x)
        # =================================
        # Mask net
        # =================================
        # [bs, 144, 1024] -> [bs, 64, 48, 48]
        bs = x.shape[0]
        x_to_mask = x.view(bs, 12, 12, 4, 4, -1).view(bs, 48, 48, 64).permute(0, 3, 1, 2)
        mask = self.mask(x_to_mask)
        
        x = self.global_pool(x.permute(0, 2, 1)).view(bs, -1)
        h = self.myfc(self.dropout(x))
        if is_mask:
            return h, mask
        else:
            return h
        


class Net605(nn.Module):
    def __init__(self, name="resnest101e"):
        super(Net605, self).__init__()
        swin = timm.create_model(name, False)

        self.patch_embed = swin.patch_embed
        self.pos_drop = swin.pos_drop
        self.b0 = swin.layers[0]
        self.b1 = swin.layers[1]
        self.b2 = swin.layers[2]
        self.b3 = swin.layers[3]
        self.norm = swin.norm

        self.global_pool = swin.avgpool
        self.dropout = nn.Dropout(0.5)
        self.myfc = nn.Linear(swin.head.in_features, 5)
        # self.enet.classifier = nn.Identity()

        # ===============================
        # Mask netfork after layer16
        # ===============================
        self.mask = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    def forward(self, x, is_mask=False):
        x = self.patch_embed(x)
        x = self.pos_drop(x)
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        # =================================
        # Mask net
        # =================================
        # [bs, 144, 1024] -> [bs, 64, 48, 48]
        bs = x.shape[0]
        x_to_mask = x.view(bs, 12, 12, 4, 4, -1).view(bs, 48, 48, 64).permute(0, 3, 1, 2)
        mask = self.mask(x_to_mask)

        x = self.b3(x)
        x = self.norm(x)
        
        x = self.global_pool(x.permute(0, 2, 1)).view(bs, -1)
        h = self.myfc(self.dropout(x))
        if is_mask:
            return h, mask
        else:
            return h




class Net606(nn.Module):
    def __init__(self, name="resnest101e"):
        super(Net606, self).__init__()
        swin = timm.create_model(name, False)

        self.patch_embed = swin.patch_embed
        self.pos_drop = swin.pos_drop
        self.b0 = swin.layers[0]
        self.b1 = swin.layers[1]
        self.b2 = swin.layers[2]
        self.b3 = swin.layers[3]
        self.norm = swin.norm

        self.global_pool = swin.avgpool
        self.dropout = nn.Dropout(0.5)
        self.myfc = nn.Linear(swin.head.in_features, 5)
        # self.enet.classifier = nn.Identity()

        # ===============================
        # Mask netfork after layer16
        # ===============================
        self.mask1 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )

        self.mask2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # # nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    def forward(self, x, is_mask=False):
        x = self.patch_embed(x)
        x = self.pos_drop(x)
        x = self.b0(x)
        x = self.b1(x)
        x = self.b2(x)
        # =================================
        # Mask net
        # =================================
        # [bs, 144, 1024] -> [bs, 64, 48, 48]
        bs = x.shape[0]
        x_to_mask = x.view(bs, 12, 12, 4, 4, -1).view(bs, 48, 48, 64).permute(0, 3, 1, 2)
        mask1 = self.mask1(x_to_mask)

        x = self.b3(x)
        x = self.norm(x)

        # =================================
        # Mask net
        # =================================
        # [bs, 144, 1024] -> [bs, 64, 48, 48]
        bs = x.shape[0]
        x_to_mask = x.view(bs, 12, 12, 4, 4, -1).view(bs, 48, 48, 64).permute(0, 3, 1, 2)
        mask2 = self.mask2(x_to_mask)
        
        x = self.global_pool(x.permute(0, 2, 1)).view(bs, -1)
        h = self.myfc(self.dropout(x))
        if is_mask:
            return h, mask1, mask2
        else:
            return h

In [ ]:
# ==========================================================
# Dataset
# ==========================================================

class CustomDataset(Dataset):
    def __init__(self,
                 df,
                 image_size_list,
                 mode="train",
                 clahe=False,
                 mix=False,
                 use_npy=True,
                 ):

        self.df = df.reset_index(drop=True)
        self.image_size_list = image_size_list
        self.transform = [albumentations.Compose([
                                albumentations.Resize(i_s, i_s),
                                ToTensorV2(p=1)
                        ]) for i_s in image_size_list]

        self.mode = mode
        self.clahe = clahe
        self.mix = mix
        if self.clahe or self.mix:
            self.clahe_transform = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(16, 16))

        self.cols = target_columns
        self.use_npy = use_npy

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]

        if self.use_npy:
            # images = np.load(row.npy_path)
            images = cv2.imread(row)
        else:
            images = pydicom.read_file(row.dicom_path).pixel_array

        if self.clahe:
            single_channel = images[:, :, 0].astype(np.uint8)
            single_channel = self.clahe_transform.apply(single_channel)
            images = np.array([
                single_channel,
                single_channel,
                single_channel
            ]).transpose(1, 2, 0)
        elif self.mix:
            single_channel = images[:, :, 0].astype(np.uint8)
            clahe_channel = self.clahe_transform.apply(single_channel)
            hist_channel = cv2.equalizeHist(single_channel)
            images = np.array([
                single_channel,
                clahe_channel,
                hist_channel
            ]).transpose(1, 2, 0)

        return_dict = {}
        for n_tf, i_s in enumerate(self.image_size_list):
            return_dict[i_s] = self.transform[n_tf](image=images)['image'] / 255
        return return_dict

In [ ]:
study_sub_list = [


    # prediction set
    [
        # backbone
        Net414("tf_efficientnetv2_m_in21k"),
        "tf_efficientnetv2_m_in21k",
        # img_size
        512,
        # weight list
        [
            "/kaggle/input/weight-siim-rsna-v1/exp414/cv0_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v1/exp414/cv1_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v1/exp414/cv2_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v1/exp414/cv3_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v1/exp414/cv4_weight_checkpoint_best.pth",
        ],
    ],

#     # prediction set
#     [
#         # backbone
#         Net415("tf_efficientnetv2_m_in21k"),
#         "tf_efficientnetv2_m_in21k",
#         # img_size
#         512,
#         # weight list
#         [
#             "/kaggle/input/weight-siim-rsna-v2/exp415/cv0_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp415/cv1_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp415/cv2_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp415/cv3_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp415/cv4_weight_checkpoint_best.pth",
#         ],
#     ],

#     # prediction set
#     [
#         # backbone
#         Net416("tf_efficientnetv2_m_in21k"),
#         "tf_efficientnetv2_m_in21k",
#         # img_size
#         512,
#         # weight list
#         [
#             "/kaggle/input/weight-siim-rsna-v2/exp416/cv0_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp416/cv1_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp416/cv2_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp416/cv3_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp416/cv4_weight_checkpoint_best.pth",
#         ],
#     ],

    # prediction set
    [
        # backbone
        Net417("tf_efficientnetv2_m_in21k"),
        "tf_efficientnetv2_m_in21k",
        # img_size
        512,
        # weight list
        [
            "/kaggle/input/weight-siim-rsna-v2/exp417/cv0_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp417/cv1_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp417/cv2_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp417/cv3_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp417/cv4_weight_checkpoint_best.pth",
        ],
    ],



    # ===========================================
    # Eff v2 L
    # ===========================================


#     # prediction set
#     [
#         # backbone
#         Net419("tf_efficientnetv2_l_in21k"),
#         "tf_efficientnetv2_l_in21k",
#         # img_size
#         512,
#         # weight list
#         [
#             "/kaggle/input/weight-siim-rsna-v2/exp419/cv0_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp419/cv1_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp419/cv2_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp419/cv3_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp419/cv4_weight_checkpoint_best.pth",
#         ],
#     ],

#     # prediction set
#     [
#         # backbone
#         Net420("tf_efficientnetv2_l_in21k"),
#         "tf_efficientnetv2_l_in21k",
#         # img_size
#         512,
#         # weight list
#         [
#             "/kaggle/input/weight-siim-rsna-v2/exp420/cv0_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp420/cv1_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp420/cv2_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp420/cv3_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp420/cv4_weight_checkpoint_best.pth",
#         ],
#     ],

    # prediction set
    [
        # backbone
        Net420("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/kaggle/input/weight-siim-rsna-v3/exp520/cv0_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp520/cv1_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp520/cv2_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp520/cv3_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp520/cv4_weight_checkpoint_best.pth",
        ],
    ],


]

In [ ]:
# =============================================================================
# predict def
# =============================================================================
def predict(model_dict, test_loader, hflip=True, scale=False, scale_hflip=False):
    pred_list1 = []
    with torch.no_grad():
        for images_all in tqdm(test_loader):

            # each model
            pred_list2 = []
            for img_size, model_list in model_dict.items():
                images = images_all[img_size].to(device)
                bs = images.shape[0]
                
                for model in model_list:
                    preds = model(images).cpu().detach()               
                    pred_list2.append(preds.sigmoid())
                
                if hflip:
                    for model in model_list:
                        preds = model(images.flip(-1)).cpu().detach()
                        pred_list2.append(preds.sigmoid())
                        
                if scale:
                    for model in model_list:
                        preds = model(F.interpolate(images, scale_factor=1.33, mode='bilinear', align_corners=False)).cpu().detach()
                        pred_list2.append(preds.sigmoid())
                        
                if scale_hflip:
                    for model in model_list:
                        preds = model(F.interpolate(images.flip(-1), scale_factor=1.33, mode='bilinear', align_corners=False)).cpu().detach()
                        pred_list2.append(preds.sigmoid())

                    
            # average prediction
            pred_list1.append(torch.stack(pred_list2, 0).mean(0))

        preds = torch.cat(pred_list1).numpy()
    # torch.cuda.empty_cache()
    return preds

In [ ]:
from copy import deepcopy

# key: image size, value: model
model_dict = {}

image_size_list = []

for model_set in tqdm(study_sub_list):
    # 画像サイズをkey, modelのリストをvalueにする
    # keyがまだない場合はからのリストを登録
    model_dict.setdefault(model_set[2], [])
    model_list = []
    for ckpt in tqdm(model_set[3]):
        model = model_set[0].to(device)
        weight = torch.load(ckpt, map_location=device)
        model.load_state_dict(weight["state_dict"])
        model.eval()
        model_list.append(deepcopy(model))
    model_dict[model_set[2]] += model_list
    image_size_list.append(model_set[2])
    
image_size_list = list(set(image_size_list))

In [ ]:
#load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    if is_public == False:
        sub_df = sub_df[~sub_df["id"].isin(pub_ids)].reset_index(drop=True)


# sub_df = sub_df[:study_len]
# test_paths = f'/kaggle/tmp/{split}/study/' + sub_df['id'] +'.png'

# まずはimageの数だけ予測する
sub_df = sub_df[study_len:]
test_paths = f'/kaggle/tmp/{split}/image/' + sub_df['id'] +'.png'



sub_df['negative'] = 0
sub_df['typical'] = 0
sub_df['indeterminate'] = 0
sub_df['atypical'] = 0

target_columns = sub_df.columns[2:]


test_loader = DataLoader(
    CustomDataset(test_paths, image_size_list),
    shuffle=False,
    batch_size=BATCH_SIZE,
    num_workers=NTHREAD,
    pin_memory=True,
)

prediction_0 = predict(model_dict, test_loader)
weight_0 = len(study_sub_list)

In [ ]:
study_sub_list = [
    
    # prediction set
    [
        # backbone
        Net418("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/kaggle/input/weight-siim-rsna-v3/exp551/cv0_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp551/cv1_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp551/cv2_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp551/cv3_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp551/cv4_weight_checkpoint_best.pth",
        ],
    ],

    # prediction set
    [
        # backbone
        Net419("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/kaggle/input/weight-siim-rsna-v3/exp552/cv0_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp552/cv1_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp552/cv2_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp552/cv3_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp552/cv4_weight_checkpoint_best.pth",
        ],
    ],

    # prediction set
    [
        # backbone
        Net553("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/kaggle/input/weight-siim-rsna-v3/exp553/cv0_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp553/cv1_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp553/cv2_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp553/cv3_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp553/cv4_weight_checkpoint_best.pth",
        ],
    ],

]

In [ ]:
# del first model_dict
del model_dict, test_loader
torch.cuda.empty_cache()

# load second model_dict
# key: image size, value: model
model_dict = {}

image_size_list = []

for model_set in tqdm(study_sub_list):
    # 画像サイズをkey, modelのリストをvalueにする
    # keyがまだない場合はからのリストを登録
    model_dict.setdefault(model_set[2], [])
    model_list = []
    for ckpt in tqdm(model_set[3]):
        model = model_set[0].to(device)
        weight = torch.load(ckpt, map_location=device)
        model.load_state_dict(weight["state_dict"])
        model.eval()
        model_list.append(deepcopy(model))
    model_dict[model_set[2]] += model_list
    image_size_list.append(model_set[2])
    
image_size_list = list(set(image_size_list))

test_loader = DataLoader(
    CustomDataset(test_paths, image_size_list),
    shuffle=False,
    batch_size=BATCH_SIZE,
    num_workers=NTHREAD,
    pin_memory=True,
)

# predict second model_dict
prediction_1 = predict(model_dict, test_loader)
weight_1 = len(study_sub_list)

In [ ]:
study_sub_list = [
    
    # ===========================================
    # Swin transformer
    # ===========================================
    
    # prediction set
    [
        # backbone
        Net604("swin_base_patch4_window12_384"),
        "swin_base_patch4_window12_384",
        # img_size
        384,
        # weight list
        [
            "/kaggle/input/weight-siim-rsna-v2/exp604/cv0_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp604/cv1_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp604/cv2_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp604/cv3_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp604/cv4_weight_checkpoint_best.pth",
        ],
    ],
    
    
    # prediction set
    [
        # backbone
        Net605("swin_base_patch4_window12_384"),
        "swin_base_patch4_window12_384",
        # img_size
        384,
        # weight list
        [
            "/kaggle/input/weight-siim-rsna-v2/exp605/cv0_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp605/cv1_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp605/cv2_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp605/cv3_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp605/cv4_weight_checkpoint_best.pth",
        ],
    ],


#     # prediction set
#     [
#         # backbone
#         Net606("swin_base_patch4_window12_384"),
#         "swin_base_patch4_window12_384",
#         # img_size
#         384,
#         # weight list
#         [
#             "/kaggle/input/weight-siim-rsna-v2/exp606/cv0_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp606/cv1_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp606/cv2_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp606/cv3_weight_checkpoint_best.pth",
#             "/kaggle/input/weight-siim-rsna-v2/exp606/cv4_weight_checkpoint_best.pth",
#         ],
#     ],

    # ===========================================
    # Pseudo labeling
    # ===========================================
    
    # prediction set
    [
        # backbone
        Net420("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/kaggle/input/weight-siim-rsna-v2/exp720/cv0_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp720/cv1_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp720/cv2_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp720/cv3_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v2/exp720/cv4_weight_checkpoint_best.pth",
        ],
    ],

    
]


# del first model_dict
del model_dict, test_loader
torch.cuda.empty_cache()

# load second model_dict
# key: image size, value: model
model_dict = {}

image_size_list = []

for model_set in tqdm(study_sub_list):
    # 画像サイズをkey, modelのリストをvalueにする
    # keyがまだない場合はからのリストを登録
    model_dict.setdefault(model_set[2], [])
    model_list = []
    for ckpt in tqdm(model_set[3]):
        model = model_set[0].to(device)
        weight = torch.load(ckpt, map_location=device)
        model.load_state_dict(weight["state_dict"])
        model.eval()
        model_list.append(deepcopy(model))
    model_dict[model_set[2]] += model_list
    image_size_list.append(model_set[2])
    
image_size_list = list(set(image_size_list))

test_loader = DataLoader(
    CustomDataset(test_paths, image_size_list),
    shuffle=False,
    batch_size=BATCH_SIZE,
    num_workers=NTHREAD,
    pin_memory=True,
)

# predict second model_dict
prediction_2 = predict(model_dict, test_loader)
weight_2 = len(study_sub_list)

In [ ]:
study_sub_list = [
    
    
    # ===========================================
    # Pseudo labeling
    # ===========================================
    
    # prediction set
    [
        # backbone
        Net418("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/kaggle/input/weight-siim-rsna-v3/exp751/cv0_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp751/cv1_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp751/cv2_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp751/cv3_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp751/cv4_weight_checkpoint_best.pth",
        ],
    ],
    
    # prediction set
    [
        # backbone
        Net419("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/kaggle/input/weight-siim-rsna-v3/exp752/cv0_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp752/cv1_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp752/cv2_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp752/cv3_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp752/cv4_weight_checkpoint_best.pth",
        ],
    ],
    
    # prediction set
    [
        # backbone
        Net553("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/kaggle/input/weight-siim-rsna-v3/exp753/cv0_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp753/cv1_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp753/cv2_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp753/cv3_weight_checkpoint_best.pth",
            "/kaggle/input/weight-siim-rsna-v3/exp753/cv4_weight_checkpoint_best.pth",
        ],
    ],

]


# del first model_dict
del model_dict, test_loader
torch.cuda.empty_cache()

# load second model_dict
# key: image size, value: model
model_dict = {}

image_size_list = []

for model_set in tqdm(study_sub_list):
    # 画像サイズをkey, modelのリストをvalueにする
    # keyがまだない場合はからのリストを登録
    model_dict.setdefault(model_set[2], [])
    model_list = []
    for ckpt in tqdm(model_set[3]):
        model = model_set[0].to(device)
        weight = torch.load(ckpt, map_location=device)
        model.load_state_dict(weight["state_dict"])
        model.eval()
        model_list.append(deepcopy(model))
    model_dict[model_set[2]] += model_list
    image_size_list.append(model_set[2])
    
image_size_list = list(set(image_size_list))

test_loader = DataLoader(
    CustomDataset(test_paths, image_size_list),
    shuffle=False,
    batch_size=BATCH_SIZE,
    num_workers=NTHREAD,
    pin_memory=True,
)

# predict second model_dict
prediction_3 = predict(model_dict, test_loader)
weight_3 = len(study_sub_list)

In [ ]:
# average prediction
weight_sum = weight_0 + weight_1 + weight_2 + weight_3
w_0 = weight_0 / weight_sum
w_1 = weight_1 / weight_sum
w_2 = weight_2 / weight_sum
w_3 = weight_3 / weight_sum

prediction = w_0 * prediction_0 + w_1 * prediction_1 + w_2 * prediction_2 + w_3 * prediction_3

prediction0 = prediction[:, :4]
prediction0_study = prediction[:, 4:5]

sub_df[target_columns] = prediction0

# study prediction

In [ ]:
# print(dict_image2study)
sub_df_temp = sub_df.copy()

# idをimage_idからstudy_idに変える
sub_df_temp["id"] = sub_df.id.map(dict_image2study)
sub_df_temp = sub_df_temp.drop('PredictionString', axis=1)
sub_df_temp = sub_df_temp.groupby('id').mean().reset_index()
sub_df_temp["PredictionString"] = "none 1 0 0 1 1"
sub_df = sub_df_temp.loc[:, ['id', 'PredictionString', 'negative', 'typical', 'indeterminate', 'atypical']]
sub_df

In [ ]:
sub_df.columns = ['id', 'PredictionString1', 'negative', 'typical', 'indeterminate', 'atypical']
df = pd.merge(df, sub_df, on = 'id', how = 'left')

for i in range(study_len):
    negative = df.loc[i,'negative']
    typical = df.loc[i,'typical']
    indeterminate = df.loc[i,'indeterminate']
    atypical = df.loc[i,'atypical']
    df.loc[i, 'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'


df_study = df[['id', 'PredictionString']]

# # if study only
# df_study.to_csv('submission.csv',index=False)
# df_study

# Image prediction, none 2 classes

In [ ]:
if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    if is_public == False:
        sub_df = sub_df[~sub_df["id"].isin(pub_ids)].reset_index(drop=True)
sub_df = sub_df[study_len:]
sub_df['none'] = 0

label_cols = sub_df.columns[2]

sub_df[label_cols] = prediction0_study
df_2class = sub_df.reset_index(drop=True)
df_2class

# Clear classification models

In [ ]:
del model_dict, test_loader

torch.cuda.empty_cache()

import gc
gc.collect()

In [ ]:
!nvidia-smi

# meta, create df
ここはmmdet, yolov5で共通

In [ ]:
meta = meta[meta['split'] == 'test']
if fast_sub:
    test_df = fast_df.copy()
else:
    test_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    if is_public == False:
        test_df = test_df[~test_df["id"].isin(pub_ids)].reset_index(drop=True)

test_df = df[study_len:].reset_index(drop=True) 
meta['image_id'] = meta['image_id'] + '_image'
meta.columns = ['id', 'dim0', 'dim1', 'split']
test_df = pd.merge(test_df, meta, on = 'id', how = 'left')

# yolov5 predict

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch
import warnings
warnings.simplefilter('ignore')

from ensemble_boxes import *


def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes


def yolo2x1y1x2y2(bboxes):
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes

def x1y1x2y2_original_size(image_height, image_width, bboxes):
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

    return bboxes

In [ ]:
dim = 512 #1024, 256, 'original'
test_dir = f'/kaggle/tmp/{split}/image'
weights_dir_list = glob('/kaggle/input/siim-rsne-weights-yolov5/*.pt')
# weights_dir = '/kaggle/input/siim-rsne-weights-yolov5/kernel_best.pt'

shutil.copytree('/kaggle/input/customized-yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5') # install dependencies

In [ ]:
for weights_dir in weights_dir_list:
    cmd = f"""python detect.py --weights {weights_dir}\
    --img 512\
    --conf 0.01\
    --iou 0.5\
    --source {test_dir}\
    --save-txt --save-conf"""
    ! {cmd}

In [ ]:
torch.cuda.empty_cache()

import gc
gc.collect()

In [ ]:
!nvidia-smi

# mmdetection

In [ ]:
import matplotlib.pyplot as plt
import torch
from glob import glob
import sys
import shutil, os
from tqdm import tqdm
import pickle


import cv2

import warnings
warnings.simplefilter('ignore')

from ensemble_boxes import *

sys.path.append('/kaggle/mmdetection/')
import mmcv
from mmdet.apis import init_detector
from mmdet.apis import inference_detector

In [ ]:
def x1y1x2y2_original_size(image_height, image_width, bboxes):
    """
    yolo => [x1, y1, x2, y1] (normalized)
    voc  => [x1, y1, x2, y2]

    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    return bboxes

In [ ]:
exp_list = ["exp006", "exp013", "exp016"]

mmdet_config = [
    {"config": f"/kaggle/input/siim-mmdet-weight/{e}/fold0.py", "weight": glob(f'/kaggle/input/siim-mmdet-weight/{e}/cv*/*.pth')} for e in exp_list
]


mmdet_model_list = []
for mmdet_dic in mmdet_config:
    cfg = mmdet_dic["config"]
    print(cfg)
    print(f"num of weight: {len(mmdet_dic['weight'])}")
    for w in mmdet_dic["weight"]:
        mmdet_model_list.append(init_detector(cfg, w, device))

In [ ]:
result_mmdet_path = f'/kaggle/tmp/mmdet_result/'
os.makedirs(result_mmdet_path, exist_ok=True)

In [ ]:
# =============================================================
# First prediction set
# =============================================================

iou_thr = 0.5

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)


image_ids = []
PredictionStrings = []

from time import time
time_wbf = []
time_mmdet_infer = []


for i in range(len(test_df)):
    row = test_df.iloc[i]
    image_id = row["id"]
    
    width = row.dim1
    height = row.dim0
    
    img_path = f'/kaggle/tmp/{split}/image/{image_id}.png'
    img = mmcv.imread(img_path)
    
    # result; xmin, ymin, xmax, ymax, conf
    boxes_list = []
    scores_list = []
    labels_list = []
    
    st = time()
    for model in mmdet_model_list:
        result = inference_detector(model, img)[0]
        result = result[result[:, 4] >= 0.01]
        boxes_list.append((result[:, :4] / 640).tolist())
        scores_list.append(result[:, 4].tolist())
        labels_list.append([1] * result.shape[0])
    time_mmdet_infer.append(time() - st)

    result = [boxes_list, scores_list, labels_list]
    with open(result_mmdet_path + f"{image_id}.pickle", mode="wb") as f:
        pickle.dump(result, f)

In [ ]:
print(f'mean time of infer: {np.mean(time_mmdet_infer)} sec / image')
print(f'total time of infer: {np.sum(time_mmdet_infer) / 60} min')

In [ ]:
del mmdet_model_list

torch.cuda.empty_cache()

import gc
gc.collect()

In [ ]:
# =============================================================
# Second prediction set
# =============================================================

exp_list = ["exp018", "exp022"]

mmdet_config = [
    {"config": f"/kaggle/input/siim-mmdet-weight/{e}/fold0.py", "weight": glob(f'/kaggle/input/siim-mmdet-weight/{e}/cv*/*.pth')} for e in exp_list
]


mmdet_model_list = []
for mmdet_dic in mmdet_config:
    cfg = mmdet_dic["config"]
    print(cfg)
    print(f"num of weight: {len(mmdet_dic['weight'])}")
    for w in mmdet_dic["weight"]:
        mmdet_model_list.append(init_detector(cfg, w, device))


image_ids = []
PredictionStrings = []

time_wbf = []
time_mmdet_infer = []


for i in range(len(test_df)):
    row = test_df.iloc[i]
    image_id = row["id"]
    
    width = row.dim1
    height = row.dim0
    
    img_path = f'/kaggle/tmp/{split}/image/{image_id}.png'
    img = mmcv.imread(img_path)
    
    # result; xmin, ymin, xmax, ymax, conf
    boxes_list = []
    scores_list = []
    labels_list = []
    
    st = time()
    for model in mmdet_model_list:
        result = inference_detector(model, img)[0]
        result = result[result[:, 4] >= 0.01]
        boxes_list.append((result[:, :4] / 640).tolist())
        scores_list.append(result[:, 4].tolist())
        labels_list.append([1] * result.shape[0])
    time_mmdet_infer.append(time() - st)
    
    # ==============================
    # Add first result
    # ==============================
    with open(result_mmdet_path + f"{image_id}.pickle", mode="rb") as f:
        result_first = pickle.load(f)
        
    boxes_list_1, scores_list_1, labels_list_1 = result_first
    boxes_list += boxes_list_1
    scores_list += scores_list_1
    labels_list += labels_list_1
        
    # ==========================================================================================
    # Get Yolov5 Prediction
    # ==========================================================================================
    for f_p in glob(f'runs/detect/*/labels/{image_id}.txt'):
        f = open(f_p, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32)
        data = data.reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        bboxes = np.round(np.concatenate((data[:, :2], yolo2x1y1x2y2(data[:, 2:])), axis =1).reshape(-1), 12)

        bboxes = bboxes.reshape(-1, 6)

        boxes_list.append(bboxes[:, 2:].tolist())
        scores_list.append(bboxes[:, 1].tolist())
        labels_list.append(bboxes[:, 0].tolist())
    
    # wbf
    st = time()
    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list,
                                                  weights=None, iou_thr=iou_thr, skip_box_thr=0.0)
    time_wbf.append(time() - st)
    
    # scoreを(1-none)倍する
    score_calib = (1 - df_2class[df_2class["id"] == image_id]["none"].values) ** 0.5
#     print(f"score_calib: {score_calib}")
#     print(f"scores[:3]: {scores[:3]}")
    scores = scores * score_calib
#     print(f"calib, scores[:3]: {scores[:3]}")
    
    boxes = x1y1x2y2_original_size(height, width, boxes)

    bboxes = np.concatenate([labels[:, None], scores[:, None], boxes], -1).reshape(-1).astype(str)

    image_ids.append(image_id)
    
    PredictionStrings.append(' '.join(bboxes))


pred_df = pd.DataFrame({'id':image_ids,
                        'PredictionString':PredictionStrings})

In [ ]:
resized_img = cv2.resize(img, (height, width))
ret = np.zeros((boxes.shape[0], 5))
ret[:, :4] = boxes
ret[:, 4] = scores

plt.figure()
plt.imshow(mmdet_model_list[0].show_result(resized_img, [ret], score_thr=0.1, thickness=15, font_size=150))
plt.show()

In [ ]:
del mmdet_model_list

torch.cuda.empty_cache()

import gc
gc.collect()

In [ ]:
test_df = test_df.drop(['PredictionString'], axis=1)
sub_df = pd.merge(test_df, pred_df, on = 'id', how = 'left').fillna("none 1 0 0 1 1")
sub_df = sub_df[['id', 'PredictionString']]

for i in range(sub_df.shape[0]):
    if sub_df.loc[i,'PredictionString'] == "none 1 0 0 1 1":
        continue
    sub_df_split = sub_df.loc[i,'PredictionString'].split()
    sub_df_list = []
    for j in range(int(len(sub_df_split) / 6)):
        sub_df_list.append('opacity')
        sub_df_list.append(sub_df_split[6 * j + 1])
        sub_df_list.append(sub_df_split[6 * j + 2])
        sub_df_list.append(sub_df_split[6 * j + 3])
        sub_df_list.append(sub_df_split[6 * j + 4])
        sub_df_list.append(sub_df_split[6 * j + 5])
    sub_df.loc[i,'PredictionString'] = ' '.join(sub_df_list)
    
try:
    sub_df['none'] = df_2class['none']
    for i in range(sub_df.shape[0]):
        if sub_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
            sub_df.loc[i,'PredictionString'] = sub_df.loc[i,'PredictionString'] + ' none ' + str(sub_df.loc[i,'none']) + ' 0 0 1 1'
except:
    df_study = df[['id', 'PredictionString']]
    df_study['PredictionString'] = ""
    print('No df_2class')
    for i in range(sub_df.shape[0]):
        if sub_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
            sub_df.loc[i,'PredictionString'] = sub_df.loc[i,'PredictionString']

sub_df = sub_df[['id', 'PredictionString']]   
df_study = df_study[:study_len]
df_study = df_study.append(sub_df).reset_index(drop=True)


if is_public == False:
    df_study = pd.concat([df_study, sub_public], ignore_index=True)

df_study.to_csv('/kaggle/working/submission.csv',index = False)  
shutil.rmtree('/kaggle/tmp/')
shutil.rmtree('/workspace/')
shutil.rmtree('/kaggle/working/yolov5')

In [ ]:
df_study

In [ ]:
!nvidia-smi